# Chapter 09 — Genetic Algorithms

- A population of candidate solutions evolves via selection, crossover, mutation.
- Fitness guides selection pressure; diversity prevents premature convergence.
- Representation matters: strings, vectors, trees each need bespoke operators.
- Probabilistic selection (roulette, tournament) balances exploration/exploitation.

Try: Raise POP or lower MUTATION to change convergence speed and diversity.

In [ ]:
import randomTARGET = "nature"POP = 20MUTATION = 0.05def random_gene():    return chr(random.randint(97, 122))def fitness(chrom):    return sum(c1 == c2 for c1, c2 in zip(chrom, TARGET))def crossover(a, b):    point = random.randrange(len(TARGET))    return a[:point] + b[point:]population = [''.join(random_gene() for _ in TARGET) for _ in range(POP)]generation = 0while True:    scored = sorted(((fitness(c), c) for c in population), reverse=True)    best_score, best = scored[0]    print(f"gen {generation}: best='{best}' score={best_score}")    if best_score == len(TARGET) or generation >= 15:        break    mating_pool = [c for score, c in scored for _ in range(score + 1)]    new_pop = []    for _ in range(POP):        parent1 = random.choice(mating_pool)        parent2 = random.choice(mating_pool)        child = ''.join(            (crossover(parent1, parent2)[i] if random.random() > MUTATION else random_gene())            for i in range(len(TARGET))        )        new_pop.append(child)    population = new_pop    generation += 1